## Imports

In [2]:
import numpy as np
import pandas as pd

## Constants

In [52]:
VOCAB_SIZE=2500

TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype='int')
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype='int')

In [5]:
sparse_train_data[-5:]

array([[5795, 2032,    0,    2],
       [5795, 2037,    0,    1],
       [5795, 2271,    0,    1],
       [5795, 2348,    0,    1],
       [5795, 2379,    0,    1]])

In [6]:
print('Nr of emails in training file ', np.unique(sparse_train_data[:,0]).size)
print('Nr of emails in test file ', np.unique(sparse_test_data[:,0]).size)

Nr of emails in training file  4013
Nr of emails in test file  1724


## Empty DataFrame

In [7]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [8]:
index_names = np.unique(sparse_train_data[:,0])

In [9]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [10]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Full matrix converter

In [11]:
def make_full_matrix(sparse_matrix, nr_words, doc_id=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_id]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [12]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 7.03 s


In [13]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model Training

In [14]:
full_train_data.CATEGORY.size

4013

In [15]:
full_train_data.CATEGORY.sum()

1248

In [16]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size

### Total Number of Words/Tokens

In [17]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
email_lengths = full_train_features.sum(axis=1)
total_wc = email_lengths.sum()
total_wc


429184

### Number of Tokens in Spam & Ham Emails

In [18]:
spam_lengths = email_lengths[full_train_data.CATEGORY ==1]
spam_lengths.shape

(1248,)

In [19]:
spam_wc = spam_lengths.sum()
spam_wc

176318

In [20]:
ham_lengths = email_lengths[full_train_data.CATEGORY ==0]
ham_lengths.shape

(2765,)

In [21]:
ham_wc = ham_lengths.sum()
nonspam_wc = ham_wc

### Summing the Tokens Occuring in Spam

In [22]:
full_train_features.shape

(4013, 2500)

In [23]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
summed_spam_token = train_spam_tokens.sum(axis=0) + 1

In [25]:
summed_spam_token

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       8
2496       1
2497       2
2498       1
2499      26
Length: 2500, dtype: int64

In [26]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()
summed_ham_token = train_ham_tokens.sum(axis=0) + 1

In [27]:
summed_ham_token.shape

(2500,)

## P(Token | Spam) - Probabily that a Token occurs given the Email is a spam mail

In [29]:
prob_tokens_spam  = summed_spam_token/ (spam_wc + VOCAB_SIZE)

In [30]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probabily that a Token occurs given the Email is a spam mail

In [33]:
prob_tokens_ham  = summed_ham_token/ (ham_wc + VOCAB_SIZE)
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that a Token Occurs

In [34]:
prob_tokens_all = full_train_features.sum(axis=0)/ total_wc
prob_tokens_all.sum()

1.0

## Saving the Trained Model

In [39]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [44]:
pd.DataFrame(sparse_test_data)

,0,1,2,3
0,8,2,1,1
1,8,3,1,4
2,8,4,1,2
3,8,5,1,1
4,8,6,1,2
...,...,...,...,...
117769,5793,2345,0,2
117770,5793,2358,0,4
117771,5793,2369,0,1
117772,5793,2399,0,3


In [46]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 3.33 s


In [48]:
full_test_features = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']

In [50]:
full_test_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,2,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,5,5,2,2,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,4,0,2,4,3,3,1,4,3,...,0,0,0,0,0,0,0,0,0,0


In [54]:
np.savetxt(TEST_TARGET_FILE,full_test_data['CATEGORY'] )

In [55]:
np.savetxt(TEST_FEATURE_MATRIX,full_test_features )

In [56]:
full_test_data['CATEGORY']

DOC_ID
8       1
12      1
14      1
15      1
17      1
       ..
5783    0
5786    0
5788    0
5792    0
5793    0
Name: CATEGORY, Length: 1724, dtype: int64

In [57]:
full_test_data.CATEGORY

DOC_ID
8       1
12      1
14      1
15      1
17      1
       ..
5783    0
5786    0
5788    0
5792    0
5793    0
Name: CATEGORY, Length: 1724, dtype: int64